- Sacar datos luz REAPI por mes (2019-2022): https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=month&geo_limit=ccaa
    - El precio en españa es el mismo para todos
- Sacar datos gas: https://data.cnmc.es/energia/mercado-mayorista-del-gas-natural/conjuntos-de-datos/evolucion-de-los-precios-por-productos

In [85]:
# Importamos las librerías que necesitamos
       
# Librerías para el tratamiento y manipulación de datos
# -----------------------------------------------------------------------
import pandas as pd             # Para manipular estructuras de datos como DataFrames
import numpy as np              # Para cálculos numéricos y arrays multidimensionales
import datetime                 # Para manejar fechas y horas
# Manejo del tiempo y generación de pausas
# -----------------------------------------------------------------------
from time import sleep          # Para pausar la ejecución del código por un tiempo determinado
import random                   # Para generar valores aleatorios, útil para espaciar solicitudes web
# Para importar módulos desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Para poder importar archivos desde carpetas fuera de la raíz del proyecto
from src.SupportApi import ( # Funciones personalizadas de soporte
    api_gas
)  
# Control de advertencias
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')  # Ignorar advertencias para una salida de consola más limpia

import requests


# Obtener Precios gas

Empezamos por obtener todos los precios del gas de 2019 a 2022

la función de abajo nos permite obtener los datos de 2019 a 2022. o en un rango de 2015 a 2023 que son los datos que se tienen hasta la fecha

In [66]:
df_gas = api_gas("2018-12-30","2022-12-01")
df_gas

_id fecha_de_negociacion       producto_de_mibgas  \
2601  2602           2018-12-31  Producto intradiario WD   
2602  2603           2018-12-31      Producto diario D+1   
2603  2604           2018-12-31     Producto mensual M+1   
2604  2605           2019-01-01  Producto intradiario WD   
2605  2606           2019-01-01      Producto diario D+1   
...    ...                  ...                      ...   
7228  7229           2022-11-29  Producto trimestral Q+1   
7229  7230           2022-11-30  Producto intradiario WD   
7230  7231           2022-11-30      Producto diario D+1   
7231  7232           2022-11-30     Producto mensual M+1   
7232  7233           2022-11-30  Producto trimestral Q+1   

      precios_de_los_productos_mibgas  
2601                            25.14  
2602                            24.28  
2603                            24.10  
2604                            25.09  
2605                            27.17  
...                               ...  
7228                           104.00  
7229                           125.35  
7230                           118.27  
7231                           122.37  
7232                           110.77  

[4632 rows x 4 columns]

Ahora vamos a dejar solo el producto de mibgas que nos interesa actualmente los productos que hay son:

In [67]:
df_gas["producto_de_mibgas"].unique()

array(['Producto intradiario WD', 'Producto diario D+1',
       'Producto mensual M+1', 'Producto mensual M+2',
       'Producto anual Y+1', 'Producto trimestral Q+1',
       'Producto trimestral Q+3', 'Producto mensual  M+3',
       'Producto trimestral Q+2'], dtype=object)

Vamos entonces a filtrar por el producto mensual M + 1, que nos va a proveer del precio del gas en el mes siguiente (Month + 1)

In [68]:
cond = df_gas["producto_de_mibgas"] == "Producto mensual M+1"
df_gas = df_gas[cond]

Lo guardamos, con objetivo de guardar nuestro progreso actual

In [69]:
df_gas.to_csv("../datos/01_PreciosGas/01_PrecioGasMensualSinLimpiar2019-2022.csv")

## Limpieza Datos Gas
Dado que hay múltiples precios en el mismo mes, nos vamos a quedar con el último precio registrado de cada mes

Primero ordenamos por fecha para asegurarnos de que están ordenados

In [70]:
df_gas = df_gas.sort_values(by="fecha_de_negociacion")

Ahora, creamos una nueva columna que nos deja solo el mes y el año

In [71]:
df_gas["anio_mes"] = df_gas["fecha_de_negociacion"].dt.to_period("M")
df_gas.sample()

_id fecha_de_negociacion    producto_de_mibgas  \
4187  4188           2020-04-03  Producto mensual M+1   

      precios_de_los_productos_mibgas anio_mes  
4187                             7.05  2020-04

Agrupamos por el último precio de cada mes y reseteamos el index, también generamos un index compatible para 

In [72]:
df_gas = df_gas.groupby("anio_mes").tail(1).reset_index(drop=True)
df_gas.index = df_gas.index + 1
df_gas.reset_index(inplace=True)
df_gas.sample()

index   _id fecha_de_negociacion    producto_de_mibgas  \
15     16  4176           2020-03-31  Producto mensual M+1   

    precios_de_los_productos_mibgas anio_mes  
15                             6.87  2020-03

Guardamos nuestro progreso y continuamos

In [73]:
df_gas.to_csv("../datos/01_PreciosGas/02_PrecioGasPorMesSinLimpiar.csv")

Ahora que ya tenemos los precios, por simplicidad, vamos a marcar correctamente a que mes y año pertenecen para poder limpiar más los datos, hacemos un offset de un mes

In [74]:
df_gas["fecha_de_negociacion"] = df_gas["fecha_de_negociacion"] + pd.DateOffset(months=1)

### Eliminamos las columnas innecesarias y generamos correctamente la columna aniomes

In [75]:
df_gas.drop(columns=["_id","producto_de_mibgas","anio_mes"],inplace=True)
df_gas.head()

index fecha_de_negociacion  precios_de_los_productos_mibgas
0      1           2019-01-31                            24.10
1      2           2019-02-28                            22.73
2      3           2019-03-28                            19.29
3      4           2019-04-29                            15.12
4      5           2019-05-30                            15.95

In [76]:
df_gas["fecha_de_negociacion"] = pd.to_datetime(df_gas["fecha_de_negociacion"]).dt.to_period("M").dt.to_timestamp()

In [79]:
df_gas = df_gas.rename(columns={"fecha_de_negociacion": "fecha",
                                "precios_de_los_productos_mibgas":"precio_gas"})
df_gas = df_gas.assign(fecha=df_gas.pop("fecha"))
df_gas.head()

index  precio_gas      fecha
0      1       24.10 2019-01-01
1      2       22.73 2019-02-01
2      3       19.29 2019-03-01
3      4       15.12 2019-04-01
4      5       15.95 2019-05-01

In [80]:
df_gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       48 non-null     int64         
 1   precio_gas  48 non-null     float64       
 2   fecha       48 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.3 KB


Guardamos como producto Final y pasamos a la siguiente API

In [82]:
df_gas.to_csv("../datos/01_PreciosGas/03_TablaPreciosGasFinal.csv")

# REData API (Red Eléctrica de España)

Vamos a obtener el precio mensual de la electricidad en España de 2019 a 2022 para ver la evolución
para ello haremos peticiones a esta url:
- https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2019-01-01T00:00&end_date=2019-01-31T23:59&time_trunc=month&geo_limit=ccaa

Donde vamos a modificar start_date y end_date en varias ocasiones para poder obtenerlo, ya que por limitación, la API solo nos da de año en año

In [97]:
url = f"https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2019-01-01T00:00&end_date=2019-12-31T23:59&time_trunc=month&geo_limit=ccaa" 

resultado = requests.get(url)
json = resultado.json()
precios = json["included"][-1]["attributes"]["values"]

In [ ]:
fechas = []
costes = []
for precio in precios:
    precios.append(precio["value"])
    fechas.append(precio["datetime"])
fechas

[]